In [6]:
import pandas as pd
import networkx as nx

In [1]:
# connect PostgreSQL
def query_agens(query):
    import psycopg2 as pg2
    from psycopg2 import Error
    from datetime import datetime
    
    start_tm = datetime.now()
    conn = pg2.connect(database="kg_news",user="consulting", host="192.168.0.68", port="15432")
    cur = conn.cursor()

    path_query = "set graph_path = realestate_news_v3;"
    cur.execute(path_query)
    cur.execute(query)
    result = cur.fetchall()
    
    cur.close()
    conn.close()
    
    end_tm = datetime.now()
    print('START: ', str(start_tm))
    print('END: ', str(end_tm))
    print('ELAP: ', str(end_tm - start_tm))
    
    return result

In [4]:
query = """match (s:noun)-[p:predicate]->(o:noun) return s.word, p.verb, o.word"""
result = query_agens(query)
data = pd.DataFrame(result, columns=['subject', 'predicate', 'object'])
#data = data.loc[:50, :]
data

START:  2020-12-31 10:05:43.043738
END:  2020-12-31 10:05:43.061759
ELAP:  0:00:00.018021


,subject,predicate,object
0,정부,발표하다,임대주택 공급
1,정부,위하다,전세시장안정
2,정부,계획이다,전세 대책 발표
3,정부,진행하다,관계부처간 협의
4,정부,옳다,대응방안 마련
...,...,...,...
964,공공임대,필수적이다,지속성
965,공공임대,필수적이다,사업성
966,공공임대,줄여야한다,정부역할
967,공공임대,지적하다,공급대책


In [7]:
# 방향성 없는 graph 생성
G = nx.Graph()

# n1, n2 고유값 node 생성
node = list(data['subject'].append(data['object']))
node = list(set(node)) # 중복제거
G.add_nodes_from(node)

for i in data.index:
    G.add_edges_from([(data.subject[i], data.object[i], {'verb':data.predicate[i]})])

In [8]:
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 920\nNumber of edges: 886\nAverage degree:   1.9261'

In [12]:
from networkx.algorithms.community.centrality import girvan_newman

comp = nx.algorithms.community.centrality.girvan_newman(G)
tuple(sorted(c) for c in next(comp))

(['부담', '아파트 갈아타기'],
 ['가격 상승분', '시세'],
 ['계약자', '전세 낀 집', '전세거래자료', '전세시세통계', '전세실거래가', '헤럴드경제', '확정일자'],
 ['11월',
  '2023년',
  '27.40%',
  '3+3년 개정안',
  '8조원',
  'DSR 관리기준',
  'DSR 기준 감소',
  'HUG',
  'KB 부동산 통계',
  '가계대출',
  '감정원 통계',
  '개인생활',
  '검찰개혁',
  '공공임대',
  '공공임대주택',
  '공급 모델',
  '공급 확대',
  '공급대책',
  '공급확대',
  '공시지가',
  '과거대책',
  '관계부처',
  '관계부처간 협의',
  '관심',
  '국민들 신음',
  '국토교통부',
  '규제강화',
  '금리탓',
  '금융위원회',
  '기존 정책',
  '기존 정책 추진',
  '기존대책',
  '기존정책',
  '기존정책 방향성',
  '다른 곳',
  '단기대책',
  '대북정책',
  '대응방안 마련',
  '대책',
  '대책 마련',
  '대책의 여지',
  '대출 규제',
  '도심',
  '도움',
  '도입',
  '로드맵',
  '매매시장 안정대책',
  '목적지',
  '무조건',
  '문재인정부',
  '미분양 주택 감소',
  '반응',
  '발표한 정책',
  '방어논리',
  '부동산 시정 안정',
  '부동산매매 수익',
  '부동산시장 안정',
  '부동산시장안정',
  '부동산정책',
  '부작용',
  '분노',
  '분통',
  '비난여론',
  '사업성',
  '상황',
  '새로운 대책',
  '서민불안',
  '설명자료',
  '세대분할',
  '속도',
  '시무 7조',
  '시장 수급',
  '시장 존중 대책',
  '시장불안정',
  '신규대책 발표',
  '신설',
  '실수요자',
  '실수요자 보호',
  '실수요자보호',
  '안심대출보증현황',
  '야당',
  '언론',
  '연봉 

In [ ]:
import itertools
G = nx.path_graph(8)
k = 2
comp = girvan_newman(G)
for communities in itertools.islice(comp, k):
    print(tuple(sorted(c) for c in communities))  